In [1]:
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
)

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)

# Either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("Any aspect on technology?")
print(response)

Ignoring wrong pointing object 261 0 (offset 0)
Ignoring wrong pointing object 296 0 (offset 0)
Ignoring wrong pointing object 343 0 (offset 0)


ValueError: 
******
Could not load OpenAI embedding model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

Consider using embed_model='local'.
Visit our documentation for more embedding options: https://docs.llamaindex.ai/en/stable/module_guides/models/embeddings.html#modules
******

## documents = SimpleDirectoryReader("data").load_data()


In [59]:
import os
import PyPDF2

# read pdf file 
pdf_file = 'data/Critical-Systems-Cybersecurity-Controls.pdf'
pdf_reader = PyPDF2.PdfReader(pdf_file)

In [60]:
pages = []

# extract text from each page and append if more than 10 words
for page in pdf_reader.pages:
    extracted_text = page.extract_text()
    if len(extracted_text) > 10:
        pages.append(extracted_text)
pages

[' Critical Systems\nCybersecurity Controls\n )CSCC – 1 : 2019 (\nSharing Indicator: White  \nDocument Classification: Open\n',
 'Disclaimer: The following controls will be \ngoverned by and implemented in accordance \nwith the laws of the Kingdom of Saudi Arabia, \nand must be subject to the exclusive jurisdiction \nof the courts of the Kingdom of Saudi Arabia. \nTherefore, the Arabic version will be the binding \nlanguage for all matters relating to the meaning \nor interpretation of this document.',
 'In the Name of Allah,\n The Most Gracious, \nThe Most Merciful',
 '6\nDocument Classification: OpenCritical Systems Cybersecurity Controls Sharing Indicator:  WhiteTraffic Light Protocol (TLP):\nThis marking protocol is widely used around the world. It has four colors (traffic lights):\n   Red – Personal, Confidential and for Intended Recipient Only\nThe recipient has no rights to share information classified in red with any person outside \nthe defined range of recipients either insid

In [70]:
for page in pages:
    print(page.replace('Document Classification: OpenCritical Systems Cybersecurity Controls Sharing Indicator', ''))

 Critical Systems
Cybersecurity Controls
 )CSCC – 1 : 2019 (
Sharing Indicator: White  
Document Classification: Open

Disclaimer: The following controls will be 
governed by and implemented in accordance 
with the laws of the Kingdom of Saudi Arabia, 
and must be subject to the exclusive jurisdiction 
of the courts of the Kingdom of Saudi Arabia. 
Therefore, the Arabic version will be the binding 
language for all matters relating to the meaning 
or interpretation of this document.
In the Name of Allah,
 The Most Gracious, 
The Most Merciful
6
:  WhiteTraffic Light Protocol (TLP):
This marking protocol is widely used around the world. It has four colors (traffic lights):
   Red – Personal, Confidential and for Intended Recipient Only
The recipient has no rights to share information classified in red with any person outside 
the defined range of recipients either inside or outside the organization.
  Amber – Restricted Sharing
The recipient may share information classified in amber onl

In [68]:
pages_text = ''.join(pages)
pages_text.replace('Document Classification: OpenCritical Systems Cybersecurity Controls Sharing Indicator', '')

' Critical Systems\nCybersecurity Controls\n )CSCC – 1 : 2019 (\nSharing Indicator: White  \nDocument Classification: Open\nDisclaimer: The following controls will be \ngoverned by and implemented in accordance \nwith the laws of the Kingdom of Saudi Arabia, \nand must be subject to the exclusive jurisdiction \nof the courts of the Kingdom of Saudi Arabia. \nTherefore, the Arabic version will be the binding \nlanguage for all matters relating to the meaning \nor interpretation of this document.In the Name of Allah,\n The Most Gracious, \nThe Most Merciful6\n:  WhiteTraffic Light Protocol (TLP):\nThis marking protocol is widely used around the world. It has four colors (traffic lights):\n   Red – Personal, Confidential and for Intended Recipient Only\nThe recipient has no rights to share information classified in red with any person outside \nthe defined range of recipients either inside or outside the organization.\n  Amber – Restricted Sharing\nThe recipient may share information clas

In [28]:
# import Document class from llamaindex
from llama_index.core import Document
Document.example()

Document(id_='c4f64336-66cc-43a9-a250-d12c630f3fdf', embedding=None, metadata={'filename': 'README.md', 'category': 'codebase'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='\nContext\nLLMs are a phenomenal piece of technology for knowledge generation and reasoning.\nThey are pre-trained on large amounts of publicly available data.\nHow do we best augment LLMs with our own private data?\nWe need a comprehensive toolkit to help perform this data augmentation for LLMs.\n\nProposed Solution\nThat\'s where LlamaIndex comes in. LlamaIndex is a "data framework" to help\nyou build LLM  apps. It provides the following tools:\n\nOffers data connectors to ingest your existing data sources and data formats\n(APIs, PDFs, docs, SQL, etc.)\nProvides ways to structure your data (indices, graphs) so that this data can be\neasily used with LLMs.

In [62]:
"""
documents = []
for i, page in enumerate(pages):
    documents.append(Document(
        text=page,
        metadata={"source": "saudi2030.pdf", "Title": "Saudi Vision 2030"}
    ))
documents
""" 

documents = Document(
    text=pages_text, metadata={"source": "saudi2030.pdf", "Title": "Saudi Vision 2030"})

## index = VectorStoreIndex.from_documents(documents)

In [63]:
from llama_index.core.node_parser import (
    SentenceSplitter,
)
node_parser = SentenceSplitter.from_defaults(chunk_size=1000)

nodes = node_parser.get_nodes_from_documents(documents=[documents])


In [64]:
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    #base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

from llama_index.core import Settings   
Settings.embed_model = ollama_embedding

In [65]:
index = VectorStoreIndex(nodes)

## query_engine = index.as_query_engine()


In [66]:
query = 'Any control on governance?'

retriever = index.as_retriever()


In [67]:
retriever.retrieve(query)

[NodeWithScore(node=TextNode(id_='02caf26c-a3d7-48fa-93b3-2a8d119313b8', embedding=None, metadata={'source': 'saudi2030.pdf', 'Title': 'Saudi Vision 2030'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='598e5156-7c34-4181-8996-e045dede388e', node_type='4', metadata={'source': 'saudi2030.pdf', 'Title': 'Saudi Vision 2030'}, hash='c9b3d26ca3b67dcd359880ef7ea37d85101cfb8bfd4210bace78aa2e94091111'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='b49c218d-cd9f-4c34-8910-3613e0d630ee', node_type='1', metadata={'source': 'saudi2030.pdf', 'Title': 'Saudi Vision 2030'}, hash='dcc1b6ded6bb8fd09b20d3748ed2337b088fd85ead0db73d3a02080dc96af059'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='a93d2335-f98d-4cc6-b4bb-ed619f405f45', node_type='1', metadata={}, hash='2091c601ac549921abf14fc83ecfcea14bbf0b08d7c7db5f8cd4b1b589f67aa7')}, metadata_template='{key}: {value}', metadata_separator='\n'

ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******

In [75]:
from llama_index.llms.ollama import Ollama

llm = Ollama(
    model="llama3.2:3b-instruct-fp16",
    #base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)
Settings.llm = llm

In [76]:
query_engine = index.as_query_engine()  
query_engine.query("Any control on governance?")

Response(response="Yes, a control under governance is Cybersecurity Strategy 1-1. It states that the organization's cybersecurity strategy must prioritize protecting its critical systems in addition to supporting compliance with related laws and regulations.", source_nodes=[NodeWithScore(node=TextNode(id_='02caf26c-a3d7-48fa-93b3-2a8d119313b8', embedding=None, metadata={'source': 'saudi2030.pdf', 'Title': 'Saudi Vision 2030'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='598e5156-7c34-4181-8996-e045dede388e', node_type='4', metadata={'source': 'saudi2030.pdf', 'Title': 'Saudi Vision 2030'}, hash='c9b3d26ca3b67dcd359880ef7ea37d85101cfb8bfd4210bace78aa2e94091111'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='b49c218d-cd9f-4c34-8910-3613e0d630ee', node_type='1', metadata={'source': 'saudi2030.pdf', 'Title': 'Saudi Vision 2030'}, hash='dcc1b6ded6bb8fd09b20d3748ed2337b088fd85ead0db73d3a